## Take Action: Select schedule to analyze, and environment

In [ ]:
start_date = "2020-11-2"
end_date = None # = now()!
env = "PAPER"  # values may be PAPER / PROD / BACKTEST

#### Imports

In [ ]:
import asyncio
import json
import math
import sys
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np
import pandas as pd
import pytz
import requests
from dateutil import parser
from IPython.display import HTML, display, Markdown
from liualgotrader.analytics import analysis
from pytz import timezone

import sklearn
from sklearn.cluster import Birch
print(f"SKLEARN Version {sklearn.__version__}")

%matplotlib inline


nest_asyncio.apply()

#### Load trading day details

In [ ]:
day_to_analyze = datetime.strptime(start_date, "%Y-%m-%d")
end_to_analyze = datetime.strptime(edn_date, "%Y-%m-%d") if end_date else datetime.now()
trades = analysis.load_trades(day_to_analyze, env, end_to_analyze)
algo_runs = analysis.load_runs(day_to_analyze, env, end_to_analyze)

## Show trading session performance

In [ ]:
symbol_name = []
counts = []
revenues = []
est = pytz.timezone("US/Eastern")
batch_ids = trades.batch_id.unique().tolist()
# batch_ids.reverse()

current_max = pd.options.display.max_rows
pd.set_option("display.max_rows", None)
for batch_id in batch_ids:
    how_was_my_day = pd.DataFrame()
    how_was_my_day["symbol"] = trades.loc[trades["batch_id"] == batch_id][
        "symbol"
    ].unique()
    how_was_my_day["revenues"] = how_was_my_day["symbol"].apply(
        lambda x: analysis.calc_batch_revenue(x, trades, batch_id)
    )
    how_was_my_day["count"] = how_was_my_day["symbol"].apply(
        lambda x: analysis.count_trades(x, trades, batch_id)
    )

    if len(algo_runs.loc[algo_runs["batch_id"] == batch_id].start_time) > 0:
        batch_time = (
            algo_runs.loc[algo_runs["batch_id"] == batch_id]
            .start_time.min()
            .tz_localize("utc")
            .astimezone(est)
        )
    else:
        continue

    env = algo_runs[algo_runs["batch_id"] == batch_id].algo_env.tolist()[0]
    win_ratio = round(
        1.0
        * len(how_was_my_day[how_was_my_day.revenues >= 0])
        / len(how_was_my_day[how_was_my_day.revenues < 0]),
        2,
    )
    revenues = round(sum(how_was_my_day["revenues"]), 2)
    traded = len(how_was_my_day)
    print(
        f"[{env}] {batch_id}\n{batch_time}\nTotal revenues=${revenues}\nTotal traded:{traded} Win/Lose ration {win_ratio}"
    )

    display(
        Markdown(f"{len(how_was_my_day[how_was_my_day.revenues >= 0])} **Winners**")
    )
    display(how_was_my_day[how_was_my_day.revenues >= 0].sort_values('revenues'))
    display(Markdown(f"{len(how_was_my_day[how_was_my_day.revenues < 0])} **Lossers**"))
    display(how_was_my_day[how_was_my_day.revenues < 0].sort_values('revenues'))

pd.set_option("display.max_rows", current_max)

##  Time analysis

In [ ]:
time_model = pd.DataFrame()

time_model["symbol"] = trades.symbol.unique()
time_model["time"] = time_model["symbol"].apply(
    lambda x: (
        pytz.utc.localize(
            pd.to_datetime(trades.loc[trades.symbol == x].tstamp.min())
        ).astimezone(est)
        - day_to_analyze.replace(hour=9, minute=30, tzinfo=est)
    ).total_seconds()
)
time_model["revenue"] = time_model["symbol"].apply(
    lambda x: sum(
        [analysis.calc_batch_revenue(x, trades, batch_id) for batch_id in batch_ids]
    )
)
time_model["algo"] = time_model["symbol"].apply(
    lambda x: trades.loc[trades.symbol == x].algo_name.unique().tolist()
)
time_model = time_model.sort_values("time")
time_model["total"] = time_model.revenue.cumsum()
time_model["reasons"] = time_model["symbol"].apply(
    lambda x: [
        json.loads(i)["sell"]['reasons']
        for i in trades.loc[trades.symbol == x].indicators.tolist()
        if json.loads(i)["sell"] and 'reasons' in json.loads(i)["sell"]
    ]
)

In [ ]:
plt.plot(time_model.time.tolist(), time_model.total.tolist())
plt.title("Accumulative Revenues")
plt.show()

In [ ]:
plt.scatter(
    time_model.time,
    time_model.revenue,
)
plt.title("Trade profit by time")
plt.show()

In [ ]:
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
symbols = trades["symbol"].unique()
for symbol in symbols:
    for algo in trades.loc[trades.symbol == symbol].algo_name.unique():
        position = 0
        for i, row in (
            trades.loc[(trades.symbol == symbol) & (trades.algo_name == algo)]
            .sort_values("tstamp")
            .iterrows()
        ):
            if not position:
                c1.append(symbol)
                c2.append(algo)
                c3.append(row["tstamp"])
                c4.append(0.0)
                c5.append(0.0)
                delta = 0.0
            position += row["qty"] * (
                (1 if row["operation"] == "sell" and row["qty"] > 0 else -1)
            )
            delta += (
                float(row["price"])
                * row["qty"]
                * (1 if row["operation"] == "sell" and row["qty"] > 0 else -1)
            )
            c4[-1] = delta
            c5[-1] = round((row["tstamp"] - c3[-1]).total_seconds(), 3)

trades_data = pd.DataFrame(
    data={"symbol": c1, "algo": c2, "when": c3, "revenue": c4, "duration": c5}
).sort_values("when")

In [ ]:
for algo in trades_data.algo.unique():
    plt.plot(
        trades_data.loc[trades_data.algo == algo].when.tolist(),
        trades_data.loc[trades_data.algo == algo].revenue.cumsum().tolist(),
        label=algo,
    )
plt.title("Accumulative Revenue by Algo")
plt.legend()
plt.show()

for algo in trades_data.algo.unique():
    plt.scatter(
        trades_data.loc[trades_data.algo == algo].when.tolist(),
        trades_data.loc[trades_data.algo == algo].revenue.tolist(),
        label=algo,
    )
plt.title("Trades Profit by Algo over time")
plt.legend()
plt.show()

### Revenue Distribution

In [ ]:
for algo in trades_data.algo.unique():
    trades_data.loc[trades_data.algo == algo].revenue.hist(bins=10, alpha=0.5, label=algo)
plt.legend()
